In [ ]:
import sqlite3
ltconn = sqlite3.connect("/content/ocr.db")
ltcursor = ltconn.cursor()
ltcursor.execute("CREATE TABLE card_data (id INTEGER PRIMARY KEY AUTOINCREMENT, company_name TEXT, card_holder TEXT, designation TEXT, mobile_number VARCHAR(50), email TEXT, website TEXT, area TEXT, city TEXT, state TEXT, pin_code VARCHAR(10), image LONGBLOB)")

In [ ]:
%%writefile app.py

#Required packages
import easyocr
import pandas as pd
import streamlit as st
from PIL import Image
import matplotlib.pyplot as plt
import sqlite3
import cv2
import re
import base64
import io

# Save card to process
def save_card(uploaded_card):
    with open(('/content/'+ uploaded_card.name), "wb") as f:
        f.write(uploaded_card.getbuffer())

# Processed image display
def display_image(image,res):
    for (bbox, text, prob) in res:
      # unpack the bounding box
        (tl, tr, br, bl) = bbox
        tl = (int(tl[0]), int(tl[1]))
        tr = (int(tr[0]), int(tr[1]))
        br = (int(br[0]), int(br[1]))
        bl = (int(bl[0]), int(bl[1]))
        cv2.rectangle(image, tl, br, (0, 255, 0), 2)
        cv2.putText(image, text, (tl[0], tl[1] - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    plt.rcParams['figure.figsize'] = (15,15)
    plt.axis('off')
    plt.imshow(image)

# Encoding binary data
def img_to_encode(filename):
    file = open(filename,'rb').read()
    file = base64.b64encode(file)
    return file

# Data Extration from processed data
def get_data(res):
    mobileno=''
    city=''
    company_name=''
    area=''
    for ind,i in enumerate(res):
        print(ind,i)
        # To get CARD HOLDER NAME
        if ind == 0:
            data["card_holder"]=i

        # To get DESIGNATION
        elif ind == 1:
            data["designation"]=i

        # To get MOBILE NUMBER
        elif "-" in i or "+" in i:
          if mobileno=='':
            mobileno=i
          else:
            mobileno=mobileno+","+i

        # To get WEBSITE_URL
        elif "www" in i.lower():
          data["website"]=i
        elif "WWW" in i:
            data["website"] = i

        # To get EMAIL ID
        elif "@" in i:
            data["email"]=i

        # To get AREA
        match_area1=re.findall('^[0-9].+, [a-zA-Z]+',i)
        match_area2=re.findall('[0-9] [a-zA-Z]+',i)
        print("area",match_area1,match_area2)
        if area=='':
          if match_area1:
            area=i.split(',')[0]
          elif match_area2:
            area=i.split(',')[0]

        # To get CITY NAME

        match_city1 = re.findall('.+St , ([a-zA-Z]+).+', i)
        match_city2 = re.findall('.+St,, ([a-zA-Z]+).+', i)
        match_city3 = re.findall('^[E].*',i)
        if city=='':
          if match_city1:
            city=match_city1[0]
          elif match_city2:
            city=match_city2[0]
          elif match_city3:
            city=match_city3[0]

        # To get STATE
        state_match1 = re.findall('[a-zA-Z]{9} +[0-9]',i)
        state_match2 = re.findall('^[0-9].+, ([a-zA-Z]+);',i)
        if state_match1:
              data["state"]=i[:9]
        elif state_match2:
            data["state"]=i.split()[-1]
        if len(data["state"])== 2:
            data["state"].pop(0)

        # To get PINCODE
        if len(i)>=6 and i.isdigit():
            data["pin_code"]=i
        elif re.findall('[a-zA-Z]{9} +[0-9]',i):
            data["pin_code"]=i[10:]


        # To get COMPANY NAME
        elif ind == len(res)-1 or ind == len(res)-2:
          if company_name=='':
            company_name=i
          else:
            company_name=company_name+ " " + i

    data["area"]=area
    data["company_name"]=company_name
    data["city"]=city
    data["mobile_number"]= mobileno
    data["image"]=img_to_encode(uploaded_card.name)
    return data

#streamlit  page setting
icon = Image.open("card.jpg")
st.set_page_config(page_title= "BizCardX - Kavitha",
                page_icon= icon,
                layout= "wide",
                initial_sidebar_state= "expanded",
                )

st.subheader(":blue[BizCardX : Extracting Business Card Data with OCR]")
tab1,tab2,tab3,tab4=st.tabs([":blue[Upload]",":blue[Modify]",":blue[Delete]",":blue[About]"])

with tab1:
    # language selection
    lngg_lst = ['en','hi','ta']
    col1,col2=st.columns([3,7])
    with col1:
        lngselect=st.selectbox("Select Lanaguage to process",lngg_lst)
    with col2:
        uploaded_card = st.file_uploader(":blue[Click Browse files to updload card]",help='Only png,jpg,jpg files are allowed',type=["png","jpeg","jpg"])

    if uploaded_card is not None:
        save_card(uploaded_card)

        # Initialize EasyOCR
        reader = easyocr.Reader([lngselect])

        col1,col2 = st.columns(2,gap="large")
        with col1:
            st.success(":blue[Uploaded image]")
            st.image(uploaded_card)
        with col2:
            with st.spinner("Image is processing"):

                # To disable file uploader deprecation warning
                st.set_option('deprecation.showPyplotGlobalUse', False)

                saved_img = "/content/" + uploaded_card.name
                image = cv2.imread(saved_img)
                res = reader.readtext(saved_img)
                st.success(":blue[Processed image]")
                st.pyplot(display_image(image,res))

                result = reader.readtext(uploaded_card.name,detail = 0,paragraph=False)

        data = {"company_name" : [],
                "card_holder" : [],
                "designation" : [],
                "mobile_number" :[],
                "email" : [],
                "website" : [],
                "area" : [],
                "city" : [],
                "state" : [],
                "pin_code" : [],
                "image" : []
              }

        data1=get_data(result)

        df = pd.DataFrame.from_dict(data1, orient='index')
        # for converting into columns
        df = df.transpose()

        st.dataframe(df, hide_index=True)

        if st.button("Save to Database"):
            mydb = sqlite3.connect("/content/ocr.db")
            mycursor = mydb.cursor()
            try:
              for i,row in df.iterrows():
                sql = "INSERT INTO card_data (company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code,image) VALUES (?,?,?,?,?,?,?,?,?,?,?)"
                mycursor.execute(sql, tuple(row))
                mydb.commit()
            except Exception as e:
              print(e)
            st.success(":blue[Uploaded to database successfully!]")

with tab2:
    try:
        mydb = sqlite3.connect("/content/ocr.db")
        mycursor = mydb.cursor()
        mycursor.execute("select card_holder from card_data")
        myrows = mycursor.fetchall()

        business_cards = {}
        for row in myrows:
            business_cards[row[0]] = row[0]
        selected_card = st.selectbox(":blue[Select card holder name to update]", list(business_cards.keys()))
        st.caption(":blue[Update or modify any data below]")
        mycursor.execute("SELECT company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code,image FROM card_data WHERE card_holder=?",
                        (selected_card,))
        result = mycursor.fetchone()

        # DISPLAYING ALL THE INFORMATIONS
        company_name = st.text_input("Company Name", result[0])
        card_holder = st.text_input("Card Holder Name", result[1])
        designation = st.text_input("Designation", result[2])
        mobile_number = st.text_input("Mobile_Number", result[3])
        email = st.text_input("Email", result[4])
        website = st.text_input("Website", result[5])
        area = st.text_input("Area", result[6])
        city = st.text_input("City", result[7])
        state = st.text_input("State", result[8])
        pin_code = st.text_input("Pin Code", result[9])

        # decode binary data to image
        image = result[10]
        binary_data = base64.b64decode(image)
        image = Image.open(io.BytesIO(binary_data))
        st.image(image)

        if st.button(":blue[Update changes to DB]"):
            # Update the information for the selected business card in the database
            mycursor.execute("""UPDATE card_data SET company_name=?,card_holder=?,designation=?,mobile_number=?,email=?,website=?,area=?,city=?,state=?,pin_code=?
                                WHERE card_holder=?""", (company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code,selected_card))
            mydb.commit()
            st.success(":blue[Changed data updated in database successfully.]")

        # display data from db
        if st.button(":blue[View data]"):
            mycursor.execute("select company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code from card_data")
            myrows = mycursor.fetchall()
            updated_df = pd.DataFrame(myrows,columns=["Company_Name","Card_Holder","Designation","Mobile_Number","Email","Website","Area","City","State","Pin_Code"])
            st.dataframe(updated_df, hide_index=True)
    except:
        st.error("There is no data available in the database")
with tab3:
      mydb = sqlite3.connect("/content/ocr.db")
      mycursor = mydb.cursor()
      mycursor.execute("SELECT card_holder FROM card_data")
      result = mycursor.fetchall()
      business_cards = {}
      for row in result:
          business_cards[row[0]] = row[0]
      selected_card = st.selectbox("Select card holder name to Delete", list(business_cards.keys()))
      st.write(f":blue[You have selected] :green[**{selected_card}'s**]:blue[ card to delete]")
      st.write(":blue[Confirm to delete this card?]")

      if st.button(":blue[Yes Delete]"):
          mycursor.execute(f"DELETE FROM card_data WHERE card_holder='{selected_card}'")
          mydb.commit()
          st.success(":blue[Business card information deleted from database.]")

with tab4:
    st.caption(":blue[Overview:]")
    st.caption(":blue[This application allows users to upload a business card image and extract relevant information from it using easyOCR.]")
    st.caption(":blue[The extracted informations are company name, card holder name, designation, mobile number, email address, website URL, area, city, state and pin code.]")
    st.caption(":blue[You can view the extracted information in the card.]")
    st.caption(":blue[Technologies :]")
    st.caption(":blue[streamlit GUI, SQL lite,OCR Data Extraction and Python]")
